In [ ]:
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_set_process)

from sklearn.model_selection import GridSearchCV

In [ ]:
#Note！ that the input data name is data_set_process
！

In [ ]:
#Basic model, not default parameters, optimized for data volume

In [ ]:
train_size = int(len(data_set_process)*0.8) #Partitioned data sets  0.8/0.2
test_size = len(data_set_process) - train_size
train_XGB, test_XGB = scaled_data[0:train_size,:],scaled_data[train_size:len(data_set_process),:]

train_XGB_X, train_XGB_Y = train_XGB[:,:(len(data_set_process.columns)-1)],train_XGB[:,(len(data_set_process.columns)-1)]
test_XGB_X, test_XGB_Y = test_XGB[:,:(len(data_set_process.columns)-1)],test_XGB[:,(len(data_set_process.columns)-1)]


In [ ]:
# Algorithm parameters
params = {
    'booster':'gbtree',
    'objective':'binary:logistic',  # 此处为回归预测，这里如果改成multi:softmax 则可以进行多分类
    'gamma':0.1,
    'max_depth':5,
    'lambda':3,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':3,
    'slient':1,
    'eta':0.1,
    'seed':1000,
    'nthread':8,
}

In [ ]:
#Generate dataset format
xgb_train = xgb.DMatrix(train_XGB_X,label = train_XGB_Y)
xgb_test = xgb.DMatrix(test_XGB_X,label = test_XGB_Y)
num_rounds = 300
watchlist = [(xgb_test,'eval'),(xgb_train,'train')]

In [ ]:
#xgboost Model Training
model_xgb = xgb.train(params,xgb_train,num_rounds,watchlist)

In [ ]:
#Make predictions on the test set
y_pred_xgb = model_xgb.predict(xgb_test)


In [ ]:
mae_xgb = mean_absolute_error(test_XGB_Y, y_pred_xgb)
mse_xgb = mean_squared_error(test_XGB_Y, y_pred_xgb)
rmse_xgb = np.sqrt(mse)
r2_xgb = r2_score(test_XGB_Y, y_pred_xgb)

print('MAE: {:.4f}'.format(mae_xgb))
print('MSE: {:.4f}'.format(mse_xgb))
print('RMSE: {:.4f}'.format(rmse_xgb))
print('R^2: {:.4f}'.format(r2_xgb))


In [ ]:
mape_xgb = np.mean(np.abs(y_pred_xgb-test_XGB_Y)/test_XGB_Y)*100
print('The average error rate of XGBoost is：{}%'.format(mape_xgb))  

In [ ]:
plt.plot(test_XGB_Y, color = 'red', label = 'Real for Test set')
plt.plot(y_pred_xgb, color = 'blue', label = 'Predicted for Test set')
plt.title('Prediction for Test set')
plt.xlabel('Time')
plt.ylabel('Predicted results')
plt.legend()
plt.show()

In [ ]:
##GridSearchCV in XGBoost

In [ ]:
# Algorithm parameters
params = {
    'booster': ['gbtree'],
    'objective': ['binary:logistic'],  # Here is the regression prediction, here if you change to multi:softmax, then you can do multiple classification
    'gamma': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'lambda': [1, 3, 5],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'min_child_weight': [1, 3, 5],
    'eta': [0.05, 0.1, 0.2],
    'seed': [1000],
    'nthread': [8],
}

In [ ]:
#Create XGBoost model
xgb_model_CV = xgb.XGBRegressor()

#GridSearchCV with 5-fold cross-validation
grid = GridSearchCV(estimator=xgb_model_CV, param_grid=params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

#Fit the training data to the model
grid.fit(train_XGB_X, train_XGB_Y)

#Best parameters and score from GridSearchCV
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

#Use the best parameters to train the model
best_params = grid.best_params_
xgb_model_best = xgb.train(best_params, xgb_train, num_rounds, watchlist)

#Make predictions on the test set
y_pred_xgb_best = xgb_model_best.predict(xgb_test)
mape_xgb_best = np.mean(np.abs(y_pred_xgb-test_XGB_Y)/test_XGB_Y)*100
print('The average error rate of XGBoost with best parameters is: {}%'.format(mape_xgb_best))

In [ ]:
mae_xgb_best = mean_absolute_error(test_XGB_Y, y_pred_xgb_best)
mse_xgb_best = mean_squared_error(test_XGB_Y, y_pred_xgb_best)
rmse_xgb_best = np.sqrt(mse)
r2_xgb_best = r2_score(test_XGB_Y, y_pred_xgb_best)


print('MAE: {:.4f}'.format(mae_xgb_best))
print('MSE: {:.4f}'.format(mse_xgb_best))
print('RMSE: {:.4f}'.format(rmse_xgb_best))
print('R^2: {:.4f}'.format(r2_xgb_best))

In [ ]:
plt.plot(test_XGB_Y, color = 'red', label = 'Real for Test set')
plt.plot(y_pred_xgb_best, color = 'blue', label = 'Predicted for Test set')
plt.title('Prediction for Test set')
plt.xlabel('Time')
plt.ylabel('Predicted results')
plt.legend()
plt.show()